# Steel Challenge Shooting Association (SCSA) Stages
## Scaled Distances for Indoor Dry Fire Training

This notebook outputs an HTML table and other associated files that scale the eight standard Steel Challenge stages to a fixed distance you specify.  The intent is to render each target in the proper percieved size for dry fire training at reasonable distances (e.g., 10 feet).  

For example, in the 'Five to Go' stage, target T1 (10" plate) is placed 30' from the baseline and 10'3" to the left of the center line.  Target T4 (10" plate) is placed 54' from the baseline and 10'3" to the right of the center line. At a training distance of 10', T1 appears as a 3.5" plate, 3.4' left of the center line, and T4 as a 1.9" plate, 1.9' right of the center line. 

The notebook creates an SVG of each target of the prescribed size according to the training distance.  Where practical, cut out these targets and afix them to a wall according to the stage diagrams for dry fire practice.  Except for Pendulum (SC-106) the height of the targets are generally all the same (at eye-level).

**Notes:**
- Each stage contains tips for shooting order.
- Targets with a "*" indicate the stop target
- In the distance HTML file, clicking the stage image will enlarge it
- Each target is saved as an SVG image to a separate file for printing

**Links**
- SCSA website - http://www.scsa.org
- Stage diagrams and tips - https://teammatchtracker.com/steel-challenge-stages



**Enter desired training distance (in feet):**

In [ ]:
# Enter training distance (feet)
training_distance = 10

**Import required Python modules**

In [ ]:
from IPython.core.display import display, HTML
import csv
import os
import numpy as np

**Read CSV files of data and load into disctionary data structures**

In [ ]:
# setup targets and stages as dictionaries for easy lookup
targets = {}
stages = {}
tips = {}
target_file = 'SCSA_Targets.csv'
stages_file = 'SCSA_Stages.csv'
tip_file = 'SCSA_Tips.csv'

# Load SCSA_Targets.csv file describing targets
# Target_Type,Diameter,Height,Width
# A,          0,       24,    18
# B,          12,      0,     0
# C,          10,      0,     0
#
# targets = 
# {'A': {'diameter': '0', 
#        'size': ['24', '18']}, 
#  'B': {'diameter': '12',
#        'size': ['0', '0']}, 
#  'C': {'diameter': '10', 
#        'size': ['0', '0']}}

print('Reading {} file...'.format(target_file))
with open(target_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if len(row) > 1 and not row[0].startswith('#'):
            targets[row[0]] = {'diameter' : row[1], 'size' : [row[2], row[3]] }
            # print(row[0], "-->", targets[row[0]])
csv_file.close()

# Load SCSA_Stages.csv file describing standard stages
# Stage_Name,           Target_Num,Target_Type,Distance,Offset,Stop
# Smoke & Hope (SC-103),T1,        A,          21,      -14,   False
# Smoke & Hope (SC-103),T2,        A,          27,      -9,    False
# Smoke & Hope (SC-103),T3,        B,          15,      0,     True
# Smoke & Hope (SC-103),T4,        A,          27,      9,     False
# Smoke & Hope (SC-103),T5,        A,          21,      14,    False                                                                                
#
# stages = 
# {'Smoke & Hope (SC-103)': 
#     {'T1': 
#         {'target_type': 'A', 
#          'distance': '21', 
#          'offset': '-14', 
#          'stop': 'False', 
#          'training_target_size': [0, 0, 0], 
#          'training_offset': 0}, 
#      'T2': 
#         {'target_type': 'A', 
#          'distance': '27', 
#          'offset': '-9', 
#          'stop': 'False', 
#          'training_target_size': [0, 0, 0], 
#          'training_offset': 0}, 
# etc.

print('Reading {} file...'.format(stages_file))
with open(stages_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if len(row) > 1 and not row[0].startswith('#'):
            if row[0] in stages:
                # update sub dict
                # print('update ',row[0])
                stages[row[0]].update({row[1]: {'target_type': row[2], 'distance': row[3], 'offset': row[4], 'stop': row[5],
                                       'training_target_size': [0, 0, 0], 'training_offset': 0 }})
            else:
                # print('new ', row[0])
                stages[row[0]] = {row[1]: {'target_type': row[2], 'distance': row[3], 'offset': row[4], 'stop': row[5],
                                       'training_target_size': [0, 0, 0], 'training_offset': 0 }}
            # print(row[0], "-->", stages[row[0]])
csv_file.close()

# Load SCSA_Tips.csv file
# Stage_Name,Tip
# SC-101,1-2-3-4-5
# SC-102,RBox:1-2-4-5-3 LBox:5-4-1-2-3
# SC-103,RH:1-2-4-5-3 LH:5-4-2-1-3
# SC-104,1-2-4-5-3
# SC-105,1-2-5-4-3
# SC-106,LR:1-2-4-5-3 RL:5-4-2-1-3
# SC-107,5-4-3-2-1
# SC-108,1-2-5-4-3                                                                                
#
# tips = 
# {'SC-101': 
#     {'tip': '1-2-3-4-5' },
# {'SC-102': 
#     {'tip': 'RBox:1-2-4-5-3 LBox:5-4-1-2-3' }
# etc.

print('Reading {} file...'.format(tip_file))
with open(tip_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if len(row) > 1 and not row[0].startswith('#'):
            tips[row[0]] = {'tip' : row[1]}
            # print(row[0], "-->", tips[row[0]])
csv_file.close()

**Calculate target sizes and offsets.  Data saved in dictionary data structures.**

In [ ]:
# Calculate new target sizes and offsets
print('Calculating training target sizes at {} feet...'.format(training_distance))

for stage in stages.keys():
    print('stage: ', stage)
    for target in stages[stage].keys():
        print(' target: ', target)
        target_type = stages[stage][target]['target_type']
        print('  type:', target_type)
        distance = float(stages[stage][target]['distance'])
        print('  distance:', distance)
        offset = float(stages[stage][target]['offset'])
        print('  offset:', offset)
        stop = stages[stage][target]['stop']
        print('  stop:', stop)
        print('  -----')

        # find shooter angle to target
        sa = np.rad2deg(np.arctan(abs(offset)/distance))
        print('  shooter angle to target:', sa)
        
        # find scaled offset
        scaled_offset = np.tan(np.deg2rad(sa)) * training_distance
        if offset < 0:
            scaled_offset = scaled_offset * -1
        stages[stage][target]['training_offset'] = scaled_offset
        print('  training offset (ft): ', scaled_offset)
        
        # find actual scaled distance from shooter to target
        c = np.sqrt(training_distance**2 + scaled_offset**2)
        print('  training actual visual distance to target (ft): ', c)
        
        # calculate percieved target sizes at training distance and update data structures
        
        if target_type == 'A':
            print('  -----')
            # height and width in feet
            height = int(targets['A']['size'][0]) / 12
            width = int(targets['A']['size'][1]) / 12
            print('  actual target height (ft): ', height)
            print('  actual target width (ft): ', width)
            
            # find visual angle to height at center of target
            va = np.rad2deg(np.arctan((height / 2) / distance))
            print('  visual angle to height: ', va)
            # find scaled height (double because only considered half in calcs)
            h = (np.tan(np.deg2rad(va)) * c) * 2
            print('  training target height (ft): ', h)
            
            # find visual angle to width at center of target
            va = np.rad2deg(np.arctan((width / 2) / distance))
            print('  visual angle to width: ', va)
            # find scaled width (double because only considered half in calcs)
            w = (np.tan(np.deg2rad(va)) * c) * 2
            print('  training target width (ft): ', w)
            
            # convert back to inches
            stages[stage][target]['training_target_size'] = [0, h * 12, w * 12]

        else:
            print('  -----')
            # diameter in feet
            diameter = (int(targets[target_type]['diameter'])) / 12
            print('  actual target diameter (ft): ', diameter)
            print('  actual target radius (ft): ', diameter/2)
            
            # find visual angle to radius at center of target
            va = np.rad2deg(np.arctan((diameter / 2) / distance))
            print('  visual angle to radius', va)
            # find scaled radius
            r = np.tan(np.deg2rad(va)) * c
            print('  scaled radius (ft): ', r)
            # convert back to inches
            stages[stage][target]['training_target_size'] = [(2 * r) * 12, 0, 0]
            
        training_target_size = stages[stage][target]['training_target_size']
        print('  training_target_size (in):', training_target_size)
        
        # stages[stage][target]['training_offset'] = (((offset/12)*training_distance)/distance)*12
        training_offset = stages[stage][target]['training_offset']
        print('  training_offset (ft):', training_offset)
    

**Generate HTML output.**

In [ ]:
# Output data as inline HTML and write to file
output = "<html><bod><h1>Steel Challenge Stages Scaled to " + str(training_distance) + " Feet</h1>"
output += "<table border=1><tr><th rowspan=2>Stage</th><th rowspan=2>Target</th><th colspan=3>Target Size<br/>(inches)</th>"
output += "<th rowspan=2>Offset<br/>(feet)</th><th rowspan=2>Training<br/>Distance<br/>(feet)</th><th rowspan=2>Stage Diagram</th></tr>"
output += "<tr><th>Diameter</th><th>Height</th><th>Width</th></tr>"

# for each stage...
for stage in stages.keys():
    
    # extract image file name from stage
    stage_id = stage[stage.find("(")+1:stage.find(")")]
    img_file = '../images/' + stage_id + '.webp'
    # print('image: ', img_file)
    
    # tip and diagram
    tip = tips[stage_id]['tip']
    output += "<tr><td>" + stage + "</td><td colspan=6>Tip: " + tip
    output += "</td><td rowspan=6><a href=" + img_file + " target=_BLANK><img src="
    output += img_file + " height=100></a></td></tr>"
    
    # target
    for target in stages[stage].keys():
        stop = stages[stage][target]['stop']
        output += "<tr><td></td><td align=center>" + target
        if stop.upper() == 'TRUE':
            output +="*"
        output += "</td>"

        # get target type and output diameter or height x width
        target_type = stages[stage][target]['target_type']
        training_target_size = stages[stage][target]['training_target_size']
        if target_type.upper() == 'A':
             output += "<th></td><td>{:.1f}</td><td>{:.1f}</td>".format(training_target_size[1], training_target_size[2])
        else:         
            output += "<td>{:.1f}</td><td></td><td></td>".format(training_target_size[0])
            
        # output the target offset    
        training_offset = float(stages[stage][target]['training_offset'])
        output += "<td>{:.1f}</td>".format(training_offset)
        
        # output training distance
        output += "<td align=center>" + str(training_distance) + "</td></tr>"
        
output += "</table><p/>"
output += "<i>(C) 2019 MSRoth</i><p/></body></html>"

# once the HTML is constructed write it to a file in a folder designated by the
# training distance
output_dir = str(training_distance) + '-feet'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
f = open('{}/SCSA_training_distance-{}.html'.format(output_dir,training_distance), 'w')
f.write(output)
f.close()

print('Wrote output to {}/SCSA_training_distance-{}.html'.format(output_dir, training_distance))
# Display results
# display(HTML(output));

**Run the following cell to generate targets as SVG**

In [ ]:
# Create SVG graphics of each target and write to individual files
px = 96  # this is a guess

# for each stage...
for stage in stages.keys():
    
    stage_id = stage[stage.find("(")+1:stage.find(")")]
    
    # target
    for target in stages[stage].keys():
        html_outer = '<table border=1><tr><th align=center>{}</th></tr><tr><td align=center>{}</td></tr>'.format(stage, target)
        svg_outer = '<tr><td align=center><svg height={} width={}>{}</svg></td></tr>'
        svg_inner = ''

        target_type = stages[stage][target]['target_type']
        training_target_size = stages[stage][target]['training_target_size']
        
        # create SVG shape depending upon target type
        if target_type.upper() == 'A':
            x = training_target_size[2]
            y = training_target_size[1]
            html_outer += '<tr><td align=center>height={:.1f}\" width={:.1f}\"</td></tr>'.format(y, x)
            svg_inner += '<rect x={} y={} width={} height={} stroke=black stroke-width=3 fill=none />'.format((x * px)/4, 
                                                                                                    (y * px)/4,
                                                                                                    float(x * px),
                                                                                                    float(y * px))
            html_outer += svg_outer.format(1.5 * float(y) * px, 1.5 * float(x) * px, svg_inner)
        else: 
            r = float(training_target_size[0]) / 2.0
            html_outer += '<tr><td align=center>diameter={:.1f}\"</td></tr>'.format(r*2.0)
            svg_inner += '<circle cx={} cy={} r={} stroke=black stroke-width=3 fill=none />'.format(1.5 * (float(r) * px),
                                                                                          (1.5 * float(r) * px),
                                                                                          (float(r) * px))
            html_outer += svg_outer.format(3 * float(r) * px, 3 * float(r) * px, svg_inner) + '</table><p/>'
            
        # output += html_outer
        
        # write each target to a file
        filename = '{}/SCSA_training_targets_{}_{}-{}.html'.format(output_dir, stage_id, target, training_distance)
        f = open(filename, 'w')
        f.write('<html>{}</html>'.format(html_outer))
        f.close()

        print('Wrote output to {}'.format(filename))
        # Display results
        # display(HTML(html_outer));

**The End.  I hope this was helpful.**